In [2]:
import torch,torchvision
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

In [3]:
train_data=pd.read_csv('train.csv',dtype = np.float32)
test_data=pd.read_csv('test.csv',dtype = np.float32)

In [14]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model,self).__init__()
        
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0)
        self.relu1=nn.ReLU()
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0)
        self.relu2=nn.ReLU()
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        
        self.fc1=nn.Linear(32*4*4,10)
        
    def forward(self,x):
        out=self.cnn1(x)
        out=self.relu1(out)
        out=self.maxpool1(out)
        
        out=self.cnn2(out)
        out=self.relu2(out)
        out=self.maxpool2(out)
        
        out = torch.flatten(out, 1)
        
        out=self.fc1(out)
        
        return out
    
targets_numpy = train_data.label.values
features_numpy = train_data.loc[:,train_data.columns != "label"].values/255 # normalization

# train test split. Size of train data is 80% and size of test data is 20%. 
features_train, features_test, targets_train, targets_test = train_test_split(features_numpy,
                                                                             targets_numpy,
                                                                             test_size = 0.2,
                                                                             random_state = 42) 

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(features_test)
targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor)     

batch_size=100
n_iters = 2500
num_epochs = n_iters / (len(features_train) / batch_size)
num_epochs = int(num_epochs)


# Pytorch train and test sets
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)


# Pytorch train and test sets
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

model=CNN_model()
error=nn.CrossEntropyLoss()
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
count=0
loss_list=0
iteration_list=0
accuracy_list=0
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        train=Variable(images.view(100,1,28,28))
        label=Variable(labels)
        optimizer.zero_grad()
        output=model(train)
        loss=error(output,label)
        loss.backward()
        optimizer.step()
        
        count+=1
        
        if(count%50==0):
            correct=0
            total=0
            for images,labels in test_loader:
                test=Variable(images.view(100,1,28,28))
                output=model(test)
                predicted=torch.max(output.data,1)[1]
                total+=len(labels)
                correct+=(predicted==labels).sum()
            accuracy=100*correct/float(total)  
#             loss_list.append(loss.data)
#             iteration_list.append(count)
#             accuracy_list.append(accuracy)
        if count % 500 == 0:
            # Print Loss
            print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data, accuracy))

Iteration: 500  Loss: 0.10371905565261841  Accuracy: 96.55952453613281 %
Iteration: 1000  Loss: 0.054508622735738754  Accuracy: 97.73809814453125 %
Iteration: 1500  Loss: 0.03702089563012123  Accuracy: 97.88095092773438 %
Iteration: 2000  Loss: 0.014446084387600422  Accuracy: 98.21428680419922 %
